In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [158]:
#importing csv dataset
df = pd.read_csv('./csv/dataset_2019_2022.csv')
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 0)
df.head()

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,price,transaction_date
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,0.99,5/10/2020
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,0.70,24/10/2020
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,1.68,18/10/2020
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,2.59,23/10/2020
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,0.60,27/10/2020


In [159]:
df.transaction_date = pd.to_datetime(df.transaction_date, format='%d/%m/%Y')
df['year'] = df.transaction_date.dt.year
df['month'] = df.transaction_date.dt.month
df['day'] = df.transaction_date.dt.day
df['loyalty'] = df['loyalty'].replace('Non-Loyal', 'Non-Loyal')
# df.columns
df.head()

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,price,transaction_date,year,month,day
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,0.99,2020-10-05,2020,10,5
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,0.70,2020-10-24,2020,10,24
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,1.68,2020-10-18,2020,10,18
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,2.59,2020-10-23,2020,10,23
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,0.60,2020-10-27,2020,10,27


In [160]:
loyalties = df.groupby(['year', 'loyalty']).size().unstack().reset_index()
loyalties

loyalty,year,First Time Buyer,Loyalist,Non-Loyal
0,2019,408,8354,13891
1,2020,429,9150,13496
2,2021,241,10463,12401
3,2022,56,3706,5155


In [162]:
# COLORS 
clr_nloy = '#e95f5f'
clr_ftb = '#6691d3'
clr_loy = '#8ed366'

In [173]:
#TODO create a graph that showcases the total revenue of each loyalties per year 
#* Preferrably line chart with 3 different lines (1 per loyalties)
#* x => year, y => num_of_customers

customers_per_loyalties = df.groupby(['year', 'loyalty']).agg(num_of_customers=('customer_id', 'count')).sort_values('loyalty', ascending=False).reset_index()

loyalty_types = [customers_per_loyalties.loyalty.unique()]

ftb_revenue = customers_per_loyalties[customers_per_loyalties.loyalty == 'First Time Buyer']
loyalist_revenue = customers_per_loyalties[customers_per_loyalties.loyalty == 'Loyalist']
nonloyal_revenue = customers_per_loyalties[customers_per_loyalties.loyalty == 'Non-Loyal']

fig = px.line(customers_per_loyalties, x='year', y='num_of_customers', color='loyalty', text='num_of_customers', markers=True, width=950, 
              title='Number of Customers', color_discrete_sequence=[clr_nloy, clr_loy, clr_ftb])
fig.update_layout(xaxis= {'tickvals': [*range(int(customers_per_loyalties['year'].min()), int(customers_per_loyalties['year'].max() + 1))]})
fig.update_traces(textposition="top right")
fig.show()


In [164]:
#TODO create a chart that would showcase the total customers and the percentage of types of loyal
#* purpose is to know the current customers

total_customers = df.groupby(['loyalty']).agg(total_customer=('customer_id', pd.Series.nunique)).reset_index()
total_count_customers = total_customers['total_customer'].sum()

fig_customers = go.Figure(data=[go.Pie(labels=total_customers['loyalty'], values=total_customers['total_customer'], hole=.4, 
                                       title=str(total_count_customers), marker_colors=[clr_ftb, clr_loy,  clr_nloy])])
fig_customers.update_layout(title_text='Total Customers', font_size=12, width=500)
fig_customers.show() 


In [165]:
#TODO create a chart/table that would showcase the top 5 most selling points for each loyalties
#* purpose to figure out which commodities can be used to convert non loyal customers to loyal customers

revenue_per_commodities = df.groupby(['loyalty', 'commodity']).agg(total_bought=('price', 'sum')).reset_index()
top_commodities_per_loyalty = pd.concat([revenue_per_commodities[revenue_per_commodities.loyalty == loyalty].sort_values('total_bought', ascending=False).head(10) for loyalty in revenue_per_commodities.loyalty.unique()])

top_commodities_in_ftb = top_commodities_per_loyalty[top_commodities_per_loyalty.loyalty == 'First Time Buyer']
top_commodities_in_prom = top_commodities_per_loyalty[top_commodities_per_loyalty.loyalty == 'Non-Loyal'] 
top_commodities_in_loy = top_commodities_per_loyalty[top_commodities_per_loyalty.loyalty == 'Loyalist'] 
top_commodities_per_loyalty

,loyalty,commodity,total_bought
15,First Time Buyer,Beef,222.31
78,First Time Buyer,Frozen meat,97.96
108,First Time Buyer,Lunch meat,93.96
151,First Time Buyer,Seafood-frozen,86.98
37,First Time Buyer,Chicken,79.81
36,First Time Buyer,Cheese,71.43
135,First Time Buyer,Pork,71.01
162,First Time Buyer,Soft drinks,70.84
48,First Time Buyer,Deli meats,69.30
147,First Time Buyer,Salad,66.77


In [166]:
#* FIRST TIME BUYER

fig_ftb = px.bar(top_commodities_in_ftb, x='commodity', y='total_bought', width=750, title='First Time Buyers', color_discrete_sequence=[clr_ftb])
fig_ftb.show()

In [167]:
#* Non-Loyal

fig_prom = px.bar(top_commodities_in_prom, x='commodity', y='total_bought', width=750, title='Non-Loyal', color_discrete_sequence=[clr_nloy])
fig_prom.show()
# top_commodities_in_prom

In [168]:
#* Loyalist
fig_loy = px.bar(top_commodities_in_loy, x='commodity', y='total_bought', width=750, title='Loyalist', color_discrete_sequence=[clr_loy])
fig_loy.show()

In [169]:
#TODO create a chart showing the times where they buy the company's products 
monthly_spent = df.groupby(['loyalty', 'month']).agg(total_sales=('price', 'sum')).reset_index()
monthly_spent['month'] = pd.to_datetime(monthly_spent['month'], format='%m').dt.month_name().str[:3]
fig_history = px.bar(monthly_spent, x='month', y='total_sales', color='loyalty', title='Total Sales per Month', width=750, color_discrete_sequence=[clr_ftb, clr_loy, clr_nloy])
fig_history.update_xaxes(dtick=1)
fig_history.show()